In [36]:
from load_SynthRad_data_2d import *
from test_vis_data import *
import torch
#data_pelvis_path=r'F:\yang_Projects\Datasets\pelvis' #zy7_OLDSERVER
data_pelvis_path=r"C:\Users\56991\Datasets\Task1\pelvis"
train_number=5
val_number=1
from monai.transforms import (
    Compose,
    EnsureChannelFirstd,
    EnsureTyped,
    LoadImaged,
    RandRotate90d,
    Resized,
    ScaleIntensityd,
    SqueezeDimd,
)
train_ds, val_ds=get_file_list(data_pelvis_path, train_number, val_number)

In [47]:
def load_volumes(train_ds, val_ds, saved_name_train=None, saved_name_val=None,resized_size=(600,400,None),ifsave=False,ifcheck=False):
    # volume-level transforms for both image and segmentation
    train_transforms = Compose(
        [
            LoadImaged(keys=["image", "label"]),
            EnsureChannelFirstd(keys=["image", "label"]),
            NormalizeIntensityd(keys=["image", "label"], nonzero=False, channel_wise=True), # z-score normalization
            ResizeWithPadOrCropd(keys=["image", "label"], spatial_size=resized_size,mode="minimum"),
            Rotate90d(keys=["image", "label"], k=3),
            DivisiblePadd(["image", "label"], 16, mode="minimum")
        ]
    )
    train_volume_ds = monai.data.CacheDataset(data=train_ds, transform=train_transforms)
    val_volume_ds = monai.data.CacheDataset(data=val_ds, transform=train_transforms)
    if ifcheck:
        # use batch_size=1 to check the volumes because the input volumes have different shapes
        train_loader = DataLoader(train_volume_ds, batch_size=1)
        # use batch_size=1 to check the volumes because the input volumes have different shapes
        val_loader = DataLoader(val_volume_ds, batch_size=1)
        train_data = monai.utils.misc.first(train_loader)
        val_data = monai.utils.misc.first(val_loader)
        print("first volume's shape: ", train_data["image"].shape, train_data["label"].shape)
        print("first volume's shape: ", val_data["image"].shape, val_data["label"].shape)
    if ifsave:
        torch.save(train_volume_ds, saved_name_train)
        torch.save(val_volume_ds, saved_name_val)
    return train_volume_ds,val_volume_ds

def load_batch_slices(train_volume_ds, val_volume_ds, train_batch_size=5,val_batch_size=1,ifcheck=True):
    patch_func = monai.data.PatchIterd(
    keys=["image", "label"],
    patch_size=(None, None, 1),  # dynamic first two dimensions
    start_pos=(0, 0, 0)
    )
    patch_transform = Compose(
        [
            SqueezeDimd(keys=["image", "label"], dim=-1),  # squeeze the last dim
        ]
    )
    # for training
    train_patch_ds = monai.data.GridPatchDataset(
        data=train_volume_ds, patch_iter=patch_func, transform=patch_transform, with_coordinates=False)
    train_loader = DataLoader(
        train_patch_ds,
        batch_size=train_batch_size,
        num_workers=2,
        pin_memory=torch.cuda.is_available(),
    )
    # for validation
    val_patch_ds = monai.data.GridPatchDataset(
        data=val_volume_ds, patch_iter=patch_func, transform=patch_transform, with_coordinates=False)
    val_loader = DataLoader(
        val_patch_ds,
        batch_size=val_batch_size,
        num_workers=2,
        pin_memory=torch.cuda.is_available(),
    )
    if ifcheck:
        train_check_data = monai.utils.misc.first(train_loader)
        print(train_check_data["image"].shape, train_check_data["label"].shape)
        val_check_data = monai.utils.misc.first(val_loader)
        print(val_check_data["image"].shape, val_check_data["label"].shape)
    return train_loader, val_loader

In [49]:
train_volume_ds, val_volume_ds = load_volumes(train_ds, val_ds,saved_name_train=None, saved_name_val=None)

Loading dataset: 100%|██████████| 1/1 [00:02<00:00,  2.39s/it]


In [50]:
train_loader_slices, val_loader_slices=load_batch_slices(train_volume_ds, val_volume_ds, train_batch_size=5,val_batch_size=1,ifcheck=True)

torch.Size([5, 1, 400, 608]) torch.Size([5, 1, 400, 608])
torch.Size([1, 1, 400, 608]) torch.Size([1, 1, 400, 608])


In [52]:
batch_num=0
for train_batches in train_loader_slices:
    batch_num+=1
print(batch_num)

154


# test transforms and inversed transforms

In [ ]:
from load_SynthRad_data_2d import *
from pix2pix_slices_net import *
#data_pelvis_path=r'C:\Users\56991\Datasets\Task1\pelvis'
data_pelvis_path=r'F:\yang_Projects\Datasets\pelvis'
batch_size=8
val_batch_size=1
train_number=50
val_number=5
val_interval=1 # every epoch output validation output and save model
batch_interval=20 # every batches output training output 
num_epochs=50

model_name='pix2pix'
learning_rate=1e-3  #1e-3
num_channels=(8,16,32,64,128) #(4,8,16,32) #(8,16,32,64,128) # notice k=(32,32,None) in DivisiblePadd
strides= (2, 2, 2, 2) # (2, 2, 2, 2) or (1, 1, 1, 1) 
loss='BCELoss' # 'BCELoss' or 'SSIMLoss'
# choose GPU 0 or 1
GPU_ID = 1
device = torch.device(f'cuda:{GPU_ID}' if torch.cuda.is_available() else 'cpu') # 0=TitanXP, 1=P5000
print(torch.cuda.get_device_name(GPU_ID))

Pix2Pixtrainer_2D=Pix2Pixtrainer(learning_rate, num_channels, strides, loss,device, model_name)
val_loader=Pix2Pixtrainer_2D.val_loader
gen=Pix2Pixtrainer_2D.gen
epoch=0
val_transforms = Pix2Pixtrainer_2D.train_transforms
with torch.no_grad():
    val_images = None
    val_labels = None
    val_output = None
    val_ssim=0
    val_ssim_sum=0
    val_step=0
    print(len(val_loader))
    for val_data in val_loader:
        val_step+=1
        val_images, val_labels = val_data["image"].to(device), val_data["label"].to(device)
        print(val_images.shape)
        roi_size = (200, 200)
        sw_batch_size = 5
        slice_inferer = SliceInferer(
            roi_size=roi_size,
            sw_batch_size=sw_batch_size,
            spatial_dim=2,  # Spatial dim to slice along is defined here
            device=device,
            padding_mode="replicate",
        )
        val_output = slice_inferer(val_images, gen)
        print(val_output.shape)

        val_ssim=calculate_ssim(val_output,val_labels)
        val_ssim_sum+=val_ssim
        Pix2Pixtrainer_2D.output_val_log(epoch+1, val_step,Pix2Pixtrainer_2D.val_log_file,val_metrices=val_ssim)
        
        val_output.applied_operations = val_labels.applied_operations
        val_output_dict = {"label": val_output[0,:,:,:,:]} # always set val_batch_size=1
        with allow_missing_keys_mode(val_transforms):
            gen_img_volume_dict=val_transforms.inverse(val_output_dict)
        gen_img_volume=gen_img_volume_dict["label"]
        
        file_name_prex=f'pred_epoch_{epoch+1}_valset_{val_step}'
        #write_nifti_volume(gen_img_volume,self.saved_img_folder,file_name_prex)
        SaveImage(output_dir=Pix2Pixtrainer_2D.saved_img_folder, \
                    output_postfix=file_name_prex,resample=True)(gen_img_volume.detach().cpu())
        
    # calculate the overall mean ssim of all validation sets
    val_ssim_overall=val_ssim/val_step
    print("val_ssim of epoch %d: %.4f" % (epoch+1,val_ssim_overall))

In [ ]:
from monai.transforms.utils import allow_missing_keys_mode
from load_SynthRad_data_2d import *
import monai
data_pelvis_path=r'C:\Users\56991\Datasets\Task1\pelvis'
batch_size=8
val_batch_size=1
train_number=1
val_number=2
train_volume_ds,val_volume_ds,train_loader,val_loader,train_transforms = \
    myslicesloader(data_pelvis_path,train_number,val_number,batch_size)

test_data=monai.utils.misc.first(val_loader)
print(test_data["image"].shape)
train_check_data=monai.utils.misc.first(train_loader)
print(train_check_data["image"].shape)
test_data_clone=test_data["label"].clone()
test_transforms = train_transforms
test_dict_data=test_data_clone[0,:,:,:,:]
print(test_dict_data.shape)
test_dict = {"label": test_dict_data}
with allow_missing_keys_mode(test_transforms):
   gen_img_volume=test_transforms.inverse(test_dict)
print(gen_img_volume["label"].shape)

# Test Save PNG

In [1]:
from load_SynthRad_data_2d import *
#data_pelvis_path=r'C:\Users\56991\Datasets\Task1\pelvis'
#data_pelvis_path=r'F:\yang_Projects\Datasets\Task1\pelvis'
data_pelvis_path=r'D:\Projects\data\Task1\pelvis'
prefix='png_zscore'
saved_img_folder=f'D:\Projects\data\Task1\{prefix}\png_images'
saved_label_folder=f'D:\Projects\data\Task1\{prefix}\png_labels'
os.makedirs(saved_img_folder,exist_ok=True)
os.makedirs(saved_label_folder,exist_ok=True)
model_name='DCGAN' # monai_pix2pix pix2pix WGAN DCGAN
batch_size=8
val_batch_size=1
train_number=2
val_number=1
train_ds,train_volume_ds=pre_dataset_for_stylegan(data_pelvis_path,
                   train_number,
                   val_number,
                   saved_img_folder,
                   saved_label_folder,
                   resized_size=(512,512,None),)

d:\Projects\Environments\torch\venv\lib\site-packages\monai\utils\deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


all files in dataset: 180


In [7]:
def get_file_list(data_pelvis_path, train_number, val_number):
    #list all files in the folder
    file_list=[i for i in os.listdir(data_pelvis_path) if 'overview' not in i]
    file_list_path=[os.path.join(data_pelvis_path,i) for i in file_list]
    #list all ct and mr files in folder
    ct_file_list=[os.path.join(j,'ct.nii.gz') for j in file_list_path]
    mr_file_list=[os.path.join(j,'cbct.nii.gz') for j in file_list_path]
    # Dict Version
    train_ds = [{'image': i, 'label': j} for i, j in zip(mr_file_list[0:train_number], ct_file_list[0:train_number])]
    val_ds = [{'image': i, 'label': j} for i, j in zip(mr_file_list[-val_number:], ct_file_list[-val_number:])]
    print('all files in dataset:',len(file_list))
    return train_ds, val_ds

def sum_slices(data_pelvis_path):
    train_ds, val_ds=get_file_list(data_pelvis_path, 0, 180)
    train_ds_2d, val_ds_2d,\
    all_slices_train,all_slices_val,\
    shape_list_train,shape_list_val = transform_datasets_to_2d(train_ds, val_ds, 
                                                            saved_name_train='./train_ds_2d.csv', 
                                                            saved_name_val='./val_ds_2d.csv', 
                                                            ifsave=False)
    print(all_slices_val)
    return all_slices_val

data_pelvis_path=r'D:\Projects\data\Task1\pelvis'
data_brain_path=r'D:\Projects\data\Task1\brain'
data_pelvis_path_task2=r'D:\Projects\data\Task2\pelvis'
data_brain_path_task2=r'D:\Projects\data\Task2\brain'
all_slices_train=sum_slices(data_brain_path_task2)

all files in dataset: 180
37064


# Test Smaller Volume

In [1]:
from load_SynthRad_data_2d import *
data_pelvis_path=r'F:\yang_Projects\Datasets\Task1\pelvis'

batch_size=1
val_batch_size=1
train_number=1
val_number=1
resized_size=(600,400,150)
div_size=(16,16,16)
ifcheck_volume=True
# volume-level transforms for both image and segmentation
train_transforms = Compose(
[
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image", "label"]),
    #ScaleIntensityd(keys=["image", "label"], minv=0.0, maxv=1.0), # min-max normalization
    NormalizeIntensityd(keys=["image", "label"], nonzero=False, channel_wise=True), # z-score normalization
    ResizeWithPadOrCropd(keys=["image", "label"], spatial_size=resized_size,mode="minimum"),
    Rotate90d(keys=["image", "label"], k=3),
    DivisiblePadd(["image", "label"], k=div_size, mode="minimum")
]
)
train_ds, val_ds = get_file_list(data_pelvis_path, 
                                train_number, 
                                val_number)
#train_volume_ds, val_volume_ds 
train_volume_ds,val_volume_ds = load_volumes(train_transforms, 
                                        train_ds, 
                                        val_ds, 
                                        ifsave=False,
                                        ifcheck=ifcheck_volume)

f:\yang_Environments\torch\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
f:\yang_Environments\torch\venv\lib\site-packages\monai\utils\deprecate_utils.py:321: FutureWarning: monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
  warn_deprecated(argname, msg, warning_category)


all files in dataset: 180
check training data:
0 1PA001 image: torch.Size([1, 1, 400, 608, 160]) label: torch.Size([1, 1, 400, 608, 160])
checked all training data.
check validation data:
0 1PC098 image: torch.Size([1, 1, 400, 608, 160]) label: torch.Size([1, 1, 400, 608, 160])
checked all validation data.


In [2]:
patch_func = monai.data.PatchIterd(
    keys=["image", "label"],
    patch_size=(None, None,40),  # dynamic first two dimensions
    start_pos=(0, 0, 0)
)

# for training
train_patch_ds = monai.data.GridPatchDataset(
    data=train_volume_ds, patch_iter=patch_func, with_coordinates=False)
train_loader = DataLoader(
    train_patch_ds,
    batch_size=batch_size,
    num_workers=2,
    pin_memory=torch.cuda.is_available(),
)

val_patch_ds = monai.data.GridPatchDataset(
    data=val_volume_ds, patch_iter=patch_func, with_coordinates=False)
# for validation
val_loader = DataLoader(
    val_patch_ds, 
    num_workers=1, 
    batch_size=val_batch_size,
    pin_memory=torch.cuda.is_available())

In [4]:
for idx, train_check_data in enumerate(train_loader):
    ds_idx = idx * batch_size
    #current_item = train_patch_ds[ds_idx]
    print('check train data:')
    print('image:', train_check_data['image'].shape, 'label:', train_check_data['label'].shape)

for idx, val_check_data in enumerate(val_loader):
    ds_idx = idx * val_batch_size
    #current_item = val_volume_ds[ds_idx]
    print('check val data:')
    print('image:', val_check_data['image'].shape, 'label:', val_check_data['label'].shape)

check train data:
image: torch.Size([1, 1, 400, 608, 40]) label: torch.Size([1, 1, 400, 608, 40])
check train data:
image: torch.Size([1, 1, 400, 608, 40]) label: torch.Size([1, 1, 400, 608, 40])
check train data:
image: torch.Size([1, 1, 400, 608, 40]) label: torch.Size([1, 1, 400, 608, 40])
check train data:
image: torch.Size([1, 1, 400, 608, 40]) label: torch.Size([1, 1, 400, 608, 40])
check val data:
image: torch.Size([1, 1, 400, 608, 40]) label: torch.Size([1, 1, 400, 608, 40])
check val data:
image: torch.Size([1, 1, 400, 608, 40]) label: torch.Size([1, 1, 400, 608, 40])
check val data:
image: torch.Size([1, 1, 400, 608, 40]) label: torch.Size([1, 1, 400, 608, 40])
check val data:
image: torch.Size([1, 1, 400, 608, 40]) label: torch.Size([1, 1, 400, 608, 40])


In [2]:
import argparse
from my_configs1 import cfg as opt

parser = argparse.ArgumentParser(description="StyleGAN pytorch implementation.")
parser.add_argument('--config', default='./configs/sample.yaml')
args = parser.parse_args()
opt.merge_from_file(args.config)

print(opt.path.saved_inference_folder)

usage: ipykernel_launcher.py [-h] [--config CONFIG]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9013 --control=9011 --hb=9010 --Session.signature_scheme="hmac-sha256" --Session.key=b"b02f2661-b93c-45e7-a82a-f7587bd6637c" --shell=9012 --transport="tcp" --iopub=9014 --f=c:\Users\zy7\AppData\Roaming\jupyter\runtime\kernel-v2-757684VP9EagSKTQ.json


SystemExit: 2